In [1]:
import sys, os
sys.path.append(os.path.abspath("/Users/DavidHolzwarth/Uni/EPFL/bachelors-thesis"))

import time
import matplotlib.pyplot as plt

import LE as le
import LP as lp
import iterative as iterative
import logger_creator

import graph_tools
import networkx as nx
import numpy as np
import pandas as pd

# Create and Explore Graph

In [ ]:
df = pd.read_csv(
    "../datasets/slashdot_zoo.matrix",
    sep=r"\s+",
    comment="%",
    header=None,
    names=["source", "target", "weight"]
)
df_clean = df[df['weight'] > 0]

print("Number of edges before cleaning:", len(df))
print("After:", len(df_clean))

slashdot_zoo_graph = nx.from_pandas_edgelist(
    df_clean,
    source="source",
    target="target",
    create_using=nx.DiGraph,
    edge_attr="weight",
)

Number of edges before cleaning: 515397
After: 392179


In [4]:
len(slashdot_zoo_graph)

69995

In [5]:
slashdot_zoo_graph_prepared = graph_tools.prepare_graph(slashdot_zoo_graph.nodes(), slashdot_zoo_graph.edges(), sink_frac=0.0)
nodes = slashdot_zoo_graph_prepared.nodes()
delegations_orig = graph_tools.nx_graph_to_dict(slashdot_zoo_graph_prepared)
delegations = graph_tools.invert_graph(delegations_orig)

2025-06-25 10:57:27,781 - INFO - Initially 69995 nodes, after collapsing terminal SCCs 68934 nodes remain. In total 498 terminal SCCs were collapsed.


In [6]:
removed_nodes = 69995 - 68934
amount_of_clos_deg_cycles = 498
print("Average number of nodes per closed delegation cycle:", removed_nodes / amount_of_clos_deg_cycles)

Average number of nodes per closed delegation cycle: 2.1305220883534135


# Resolve and Explore Powers

In [7]:
powers, sinks = le.resolve_delegations(delegations, nodes)


In [8]:
powers_df = pd.DataFrame.from_dict(powers, orient="index", columns=["power"])
powers_df.describe()

,power
count,68934.000000
mean,1.000000
std,7.163889
min,0.000000
25%,0.000000
50%,0.000000
75%,1.221106
max,971.822033


In [9]:
display(powers_df.sort_values(by="power", ascending=False))

,power
107,971.822033
lost,805.363480
238,795.647896
277,432.520732
2146,374.348367
...,...
32267,0.000000
32269,0.000000
32270,0.000000
32272,0.000000


In [ ]:
print("Number of sinks:", len(sinks))
print("Total number of nodes:", len(nodes))
print("Fraction of sinks:", len(sinks) / len(nodes))

In [ ]:
plt.hist(powers_df['power'], bins=50)
plt.xlabel('Power')
plt.ylabel('Number of Nodes')
plt.title('Histogram of Node Powers')
plt.yscale('log')
plt.show()

# Benchmark

In [ ]:
logger, handler = logger_creator.create_logger("benchmarking")

# Verify the solutions are equal
powers_le, sinks_le = lp.resolve_delegations(delegations, nodes)
powers_lp, sinks_lp = lp.resolve_delegations(delegations, nodes)
powers_iter = iterative.iterate_delegations(delegations, nodes)

if powers_le != powers_lp:
    logger.error("Powers do not match")
    logger.error(f"LE: {powers_le}")
    logger.error(f"LP: {powers_lp}")
    print("Mismatching powers in LE and LP. Check the log file for details.")
    
for node in sinks_le:
    if (powers_lp[node] - powers_iter[node] < 0.01):
        continue
    else: 
        logger.error("Powers do not match")
        logger.error(f"LP: {powers_lp}")
        logger.error(f"Iterative: {powers_iter}")
        print("Mismatching powers in LP and Iterative. Check the log file for details.")
        break

n = 2

# LE
le_model = le.set_up(delegations, nodes)
start_time = time.time()
for _ in range(n):
    le.solve(*le_model)
end_time = time.time()
        
time_le = (end_time - start_time) / n

# LP
lp_model = lp.set_up(delegations, nodes)
start_time = time.time()
for _ in range(n):
    lp.solve(*lp_model)
end_time = time.time()

time_lp = (end_time - start_time) / n

# Iterative
start_time = time.time()
for _ in range(n):
    p = iterative.iterate_delegations(delegations, nodes)
end_time = time.time()

time_iterative = (end_time - start_time) / n


logger.removeHandler(handler)
handler.close()

plt.figure(figsize=(6,4))
plt.bar(["Iterative Solver", "LS Solver", "LP Solver"], [time_iterative, time_le, time_lp])
plt.ylabel("Runtime (s)")
plt.title("Algorithm Runtimes on Slashdot Zoo Graph")
plt.show()